In [1]:
# Run the stored data from the Data_Preparation.ipynb
%run ..//Data_Preparation.ipynb

Data types in 'indu' dataframe:
Time       datetime64[ns]
indu_Q1           float64
indu_Q2           float64
indu_Q3           float64
indu_Q4           float64
indu_Q5           float64
indu_Q7           float64
dtype: object

Data types in 'cons' dataframe:
Time        datetime64[ns]
cons_Q1            float64
cons_Q2            float64
cons_Q3            float64
cons_Q4            float64
cons_Q5            float64
cons_Q6            float64
cons_Q7            float64
cons_Q8            float64
cons_Q9            float64
cons_Q10           float64
cons_Q11           float64
cons_Q12           float64
dtype: object

Data types in 'gdp' dataframe:
Time     datetime64[ns]
Value           float64
dtype: object
indu_Q1           float64
indu_Q2           float64
indu_Q3           float64
indu_Q4           float64
indu_Q5           float64
indu_Q7           float64
Time       datetime64[ns]
dtype: object
cons_Q1            float64
cons_Q2            float64
cons_Q3            float64
con

In [2]:
#Split the data into training and testing data
indu_train = indu[(indu['Time'] >= '1990-01-01') & (indu['Time'] < '2020-01-01')]
cons_train = cons[(cons['Time'] >= '1990-01-01') & (cons['Time'] < '2020-01-01')]
gdp_train = gdp[(gdp['Time'] >= '1990-01-01') & (gdp['Time'] < '2020-01-01')]

indu_test = indu[(indu['Time'] >= '2020-01-01') & (indu['Time'] < '2024-01-01')]
cons_test = cons[(cons['Time'] >= '2020-01-01') & (cons['Time'] < '2024-01-01')]
gdp_test = gdp[(gdp['Time'] >= '2020-01-01') & (gdp['Time'] < '2024-01-01')]

In [3]:
#Importing the Mistral Large Language Model
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
model_name = "mistralai/Mistral-7B-v0.1"

#Defining what device to use
device = torch.device("cpu")

model = AutoModelForCausalLM.from_pretrained(model_name, torchscript=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards: 100%|██████████| 2/2 [02:13<00:00, 66.55s/it]


In [9]:
# Format the training data as a string
gdp_train_str = gdp_train.to_string(index=False)

# Inserting gdp_train into the prompt string
prompt = f"""
You are a time series forecasting model designed to predict the Danish GDP based on historical data. 
Your task is to forecast the GDP from 2020Q1 until 2023Q4, using the provided GDP training data.

The provided GDP training data is as follows:
{gdp_train_str}

Please analyze the provided data, which represents the GDP values over time, and generate predictions for the GDP for the specified timeframe. Your output should be a list of predicted GDP values for each quarter from 2020Q1 to 2023Q4.

Note: Your predictions should consider the underlying patterns in the data and provide realistic forecasts.
"""

print(prompt)


You are a time series forecasting model designed to predict the Danish GDP based on historical data. 
Your task is to forecast the GDP from 2020Q1 until 2023Q4, using the provided GDP training data.

The provided GDP training data is as follows:
      Time  Value
1990-01-01  210.2
1990-04-01  218.1
1990-07-01  209.5
1990-10-01  217.8
1991-01-01  220.2
1991-04-01  226.1
1991-07-01  219.6
1991-10-01  224.6
1992-01-01  228.1
1992-04-01  231.1
1992-07-01  227.1
1992-10-01  236.7
1993-01-01  227.8
1993-04-01  232.7
1993-07-01  227.8
1993-10-01  240.2
1994-01-01  238.3
1994-04-01  252.4
1994-07-01  242.8
1994-10-01  259.8
1995-01-01  254.3
1995-04-01  261.5
1995-07-01  252.8
1995-10-01  267.8
1996-01-01  260.8
1996-04-01  275.2
1996-07-01  269.6
1996-10-01  282.5
1997-01-01  273.9
1997-04-01  291.8
1997-07-01  281.8
1997-10-01  298.6
1998-01-01  288.4
1998-04-01  294.3
1998-07-01  293.8
1998-10-01  309.5
1999-01-01  296.3
1999-04-01  310.0
1999-07-01  306.6
1999-10-01  328.6
2000-01-01  316

In [13]:
# Define constants
max_input_length = 512
max_output_length = 512
max_new_tokens = 50
num_sequences = 1

# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the prompt
inputs = tokenizer.batch_encode_plus([prompt], return_tensors="pt", pad_to_max_length=True, max_length=max_input_length)

# Move the input to the appropriate device
model_input = inputs.to(device)

# Move the model to the appropriate device
model.to(device)

# Generate predictions
try:
    with torch.no_grad():
        output = model.generate(model_input, max_length=max_input_length + max_new_tokens, num_return_sequences=num_sequences, pad_token_id=tokenizer.eos_token_id)
except Exception as e:
    print(f"Error occurred during generation: {e}")
    output = None

# Decode and print the output
if output is not None:
    predicted_gdp = tokenizer.decode(output[0], skip_special_tokens=True)  # Assuming num_sequences = 1
    print("Predicted GDP:", predicted_gdp)



KeyboardInterrupt: 

In [ ]:
print(prompt)
print(inputs)
print(output)
print(output_decoded)